In [ ]:
import os
import shutil
import datetime
from ansys.edb.database import Database
from ansys.edb.definition import DieType, DieOrientation, SolderballShape
from ansys.edb.geometry import ExtentType
from ansys.edb.session import launch_session
from ansys.edb.definition import PadType
from ansys.edb.simulation_setup.simulation_setup import SimulationSetupType
from ansys.edb.terminal.terminals import PadstackInstanceTerminal

In [ ]:
RPC_SERVER = r"C:\Program Files\AnsysEM\v231\Win64"
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT, "data", "edb", "Galileo_edb.aedb")

In [ ]:
"""Copy original EDB folder into unique one."""

target_path = os.path.join(ROOT, "test_{}.aedb".format(str(datetime.datetime.now().date())))
shutil.copytree(EDB_FILE, target_path)

In [ ]:
"""Launch a session. Make sure old one has been disconnected prior to starting a new one, if any"""

session = launch_session(RPC_SERVER, 50051)

In [ ]:
"""Open EDB"""

edb = Database.open(target_path, False)

In [ ]:
"""Getting Layout"""

layout = edb.circuit_cells[0].layout

In [ ]:
"""Selecting 2 components from the layout"""

edb_components = [cmp for cmp in edb.circuit_cells[0].layout.groups if cmp.name in ["U2A5", "U1B5"]]

In [ ]:
"""Selecting the nets to be included for the simulation"""

included_nets = ["M_DQ<0>", "M_DQ<1>", "M_DQ<2>", "M_DQ<3>", "M_DQ<4>", "M_DQ<5>", "M_DQ<6>", "M_DQ<7>"]

In [ ]:
"""Create solders balls and coaxial ports on selected components for HFSS 3D layout simulation."""

for cmp in edb_components:
    pins = [pin for pin in cmp.members if pin.net.name in included_nets]
    for pin in pins:
        try:
            port_name = "{}_{}".format(cmp.name, pin.net.name)
            PadstackInstanceTerminal.create(layout, port_name, pin, cmp.placement_layer, pin.net)
        except:
            print("failed to create port on pin {}".format(pin.name))

    # solder ball property
    sold_prop = cmp.component_property.solder_ball_property
    sold_prop.shape = SolderballShape.SOLDERBALL_CYLINDER
    solderball_diameter = pins[0].padstack_def.data.get_pad_parameters(pins[0].padstack_def.data.layer_names[0],
                                                                            PadType.REGULAR_PAD)[1][0].value
    sold_prop.set_diameter(solderball_diameter, solderball_diameter)
    sold_prop.height = solderball_diameter
    # die property
    die_prop = cmp.component_property.die_property
    die_prop.die_type = DieType.FLIPCHIP
    die_prop.die_orientation = DieOrientation.CHIP_DOWN
    # port property
    port_prop = cmp.component_property.port_property
    port_prop.reference_size_auto = True
    # setters
    component_property = cmp.component_property
    component_property.solder_ball_property = sold_prop
    component_property.die_property = die_prop
    component_property.port_property = port_prop
    cmp.component_property = component_property

In [ ]:
"""Getting conformal extend based on net selection to be used for layout clipping."""

included_nets = [net for net in layout.nets if net.name in included_nets]
clipped_net = [net for net in layout.nets if net.name == "GND"]
expanded_extent = layout.expanded_extent(nets=included_nets, extent=ExtentType.CONFORMING, expansion_factor=0.01, expansion_unitless=False, use_round_corner=True, num_increments=1)

In [ ]:
"""Clipping layout"""

cutout_cell = layout.cell.cutout(included_nets=included_nets + clipped_net, clipped_nets=clipped_net, clipping_polygon=expanded_extent, clean_clipping=True)

In [ ]:
"""Deleting original Cell"""

edb.circuit_cells[0].delete()

In [ ]:
"""Saving EDB"""

edb.save()

In [ ]:
"""Disconnect the session"""

session.disconnect()